In [1]:
%reload_ext autoreload
%autoreload 2

## HyperParameters

### Paths


In [2]:
GATE_FOLDER = "/home/alencga1/Téléchargements/Apps/gate-developer-9.0.1-distro/gate-developer-9.0.1"
TF_IDF_FOLDER = '/home/alencga1/Documents/Gitfolders/condor-datastore/TermRaiderResults/11-19/TfidfTermbanktfidfTermBank.csv'
FOLDER_WITH_TEXTS = '/home/alencga1/Documents/Gitfolders/condor-datastore/EclipseDatastore/Datatext/11-19'


### CONSTANTS

In [3]:
TF_IDF_THRESHOLD = 0
TOP_N_TERMS = 810
PRE_TRAINED_BERT_MODEL = 'bert-base-uncased'
CLUSTERING_METHOD='complete'
CLUSTERING_METRIC ='cosine'
HIERARCHY_IMAGE_NAME = 'dendogram-11-19.png'

## Initialization

### Libs

In [4]:
# Default
import json
import unicodedata

# AI and Math
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch
import glob

# NLP
from gatenlp import Document
from gatenlp.processing.tokenizer import NLTKTokenizer
from nltk.tokenize import TreebankWordTokenizer
from gatenlp.gateworker import GateWorker
from gatenlp.processing.executor import SerialCorpusExecutor
from gatenlp.gateworker import GateWorkerAnnotator

import codecs
import csv
from functools import partial
import gatenlp
import sys
from tqdm import tqdm

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

In [5]:
# Check Versions
print("NB last updated with gatenlp version", gatenlp.__version__)
sys.path.append("/home/alencga1/anaconda3/lib/python3.9/site-packages/")
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

NB last updated with gatenlp version 1.0.8


2023-04-20 13:24:51,976|INFO|pytorch_pretrained_bert.modeling|Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [6]:
from nltk import tokenize
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package punkt to /home/alencga1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alencga1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# Setting up home variables
import os
os.environ["GATE_HOME"] = GATE_FOLDER

### Custom Libs

In [8]:
from lib.ontologyextraction.schema import Term, Concept, Concept_Taxonomy
from lib.ontologyextraction.termenrichment import termEnrichment
from lib.ontologyextraction.helpers import find_sub_list, getTokenVecs, getWordEmbeddingFromPhrase, count_clusters, create_tree

## Term Extraction


### Data Pre-processing


In [9]:
df = pd.read_csv(TF_IDF_FOLDER, sep=',', 
            quoting=csv.QUOTE_NONE, encoding='utf8')
# Delete duplicate rows based on specific columns 
df2 = df.drop_duplicates(subset=["Term"], keep='first')

In [10]:
threshold = TF_IDF_THRESHOLD
first_n = TOP_N_TERMS
# list_terms_string = list(set(df.loc[df['tfIdf'] > threshold]['Term'].iloc[:min(first_n,df.shape[0])].values))
list_terms_string = list(df2.loc[df2['tfIdf'] > threshold]['Term'].values)
list_terms_string = list_terms_string[:min(first_n,len(list_terms_string))]

In [11]:
list_of_terms = [Term(term_string, df2.loc[df2['Term'] == term_string]['tfIdf'].iloc[0]) for term_string in list_terms_string]

## Concept

### Concept Embedding

In [12]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_BERT_MODEL)

2023-04-20 13:24:52,995|INFO|pytorch_pretrained_bert.tokenization|loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/alencga1/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [13]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained(PRE_TRAINED_BERT_MODEL)

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

2023-04-20 13:24:53,502|INFO|pytorch_pretrained_bert.modeling|loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/alencga1/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2023-04-20 13:24:53,504|INFO|pytorch_pretrained_bert.modeling|extracting archive file /home/alencga1/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpnw766c0b
2023-04-20 13:24:56,011|INFO|pytorch_pretrained_bert.modeling|Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "voca

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (den

In [14]:
BERT_EMBEDDING_APP = partial(getWordEmbeddingFromPhrase, df, FOLDER_WITH_TEXTS, tokenizer,model, True, 5)

In [15]:
for term in list_of_terms:
    print(term.term_name)
    term._Apply_Embedding(BERT_EMBEDDING_APP)

code


100%|██████████| 1/1 [00:03<00:00,  3.12s/it]


nuts


100%|██████████| 1/1 [00:02<00:00,  2.75s/it]


contract


...

100%|██████████| 1/1 [00:00<00:00, 29.41it/s]


response


100%|██████████| 1/1 [00:00<00:00, 15.38it/s]


carbonization


100%|██████████| 1/1 [00:00<00:00, 25.82it/s]


In [16]:
# Find only words with actual vectors
X = np.array(list_of_terms[0].term_embedding.tolist())
labellist = []
filtered_list_of_terms = []
labellist.append(list_of_terms[0].term_name)
filtered_list_of_terms.append(list_of_terms[0])
for i in range(1, len(list_of_terms)):
    term = list_of_terms[i]
    if term.term_embedding is not None: 
        x = np.array(term.term_embedding.tolist())
        X = np.vstack((X,x))
        labellist.append(term.term_name)
        filtered_list_of_terms.append(term)
    else:
        print(term.term_name)

fund
option
month
variant
european union fund
antibody
product
detection of multiple pathogen in a single tube
multiple pathogen in a single tube
instrument
pair
OC43
fluorophore
request
cell
hunter
marburg virus
population
researcher
symptom
ebola and marburg virus
bat hunter
sample collection tube
determinant
result
pcr product
d-dioxolane nucleoside
nucleoside
minus
reagent for the purification of polymerase chain reaction product
unit
entity
minus spin column
sialic acid
legal entity penal registry
setting
column
emerging virus
necessary solution
change
bordetellum
author
test
organ
additional author for this research
additional author
discovery
fact-based news of research and discovery
kilometre
factor
reported by researcher at university
blood serum sample
newsrx journalist
datum
reactive antibody
amplifying host
deadly disease
increased interaction with human
transmission of these virus from animal
specific part
specific region
most antibody
bat cohabit
infection without symptom

## Concept Hierarchy

### Hierachical Clustering

In [17]:
plt.figure(figsize=(100, 120))
Z = sch.linkage(X, method=CLUSTERING_METHOD, metric=CLUSTERING_METRIC)
dendrogram = sch.dendrogram(Z, labels=labellist)
plt.savefig(HIERARCHY_IMAGE_NAME)
plt.show()

In [18]:
count_clusters(dendrogram)

46

### Agglomerative Clustering Example

In [19]:
cluster = AgglomerativeClustering(n_clusters=count_clusters(dendrogram), metric=CLUSTERING_METRIC,linkage=CLUSTERING_METHOD)
cluster.fit(X)

AgglomerativeClustering(linkage='complete', metric='cosine', n_clusters=46)

In [20]:
for i in range(len(cluster.labels_)):
    for term in list_of_terms:
        if term.term_name == labellist[i]:
            term.cluster = cluster.labels_[i]

In [21]:
for j in range(0,count_clusters(dendrogram)):
    print('Cluster : ' + str(j))
    for i in range(len(list_of_terms)):
        if list_of_terms[i].cluster == j:
            print('Term : ' + list_of_terms[i].term_name)

Cluster : 0
Term : http
Term : doi
Term : org
Term : html
Cluster : 1
Term : code
Term : nuts
Term : address
Term : telephone
Term : internet
Cluster : 2
Term : faculty
Term : biological sciences
Term : division
Term : biological
Term : sciences
Term : dept
Term : scientific research
Term : scientific
Cluster : 3
Term : kidney
Term : tissue
Term : serum
Term : blood
Term : animal
Term : kidney tissue
Term : saliva
Term : excreta
Term : other wildlife
Term : livestock
Term : source of food and traditional medicine
Term : cohabit
Term : wildlife
Term : traditional medicine
Term : food
Term : world
Term : worldwide
Term : continued encroachment of civilization
Term : consequence of the continued encroachment of civilization
Cluster : 4
Term : description
Term : duration
Term : performance
Term : title
Term : value
Term : place
Term : project
Term : programme
Term : procedure
Term : method
Term : notice
Term : date
Term : authority
Term : activity
Term : review
Term : body
Term : main
Term

In [22]:
for k in range(len(labellist)):
    if labellist[k] == 'address':
        print(k)

161


In [23]:
print(labellist[583])

internet


In [24]:
for k in range(len(labellist)):
    if labellist[k] == 'HIV-2':
        print(k)

48


In [25]:
with np.printoptions(threshold=np.inf):
    print(Z)

[[2.99000000e+02 3.44000000e+02 2.29578607e-03 2.00000000e+00]
 [3.95000000e+02 4.64000000e+02 4.14922809e-03 2.00000000e+00]
 [1.52000000e+02 1.65000000e+02 1.10126869e-02 2.00000000e+00]
 [1.71000000e+02 1.74000000e+02 1.32114043e-02 2.00000000e+00]
 [4.94000000e+02 6.72000000e+02 1.43018279e-02 3.00000000e+00]
 [2.54000000e+02 2.60000000e+02 1.70092500e-02 2.00000000e+00]
 [2.36000000e+02 6.71000000e+02 1.73497282e-02 3.00000000e+00]
 [5.11000000e+02 6.03000000e+02 1.74632275e-02 2.00000000e+00]
 [7.10000000e+01 1.64000000e+02 1.81659660e-02 2.00000000e+00]
 [6.10000000e+01 8.30000000e+01 1.87637186e-02 2.00000000e+00]
 [1.95000000e+02 1.97000000e+02 1.94293198e-02 2.00000000e+00]
 [3.01000000e+02 3.49000000e+02 2.05912291e-02 2.00000000e+00]
 [1.10000000e+01 2.13000000e+02 2.05920342e-02 2.00000000e+00]
 [3.82000000e+02 3.87000000e+02 2.14651204e-02 2.00000000e+00]
 [5.35000000e+02 5.82000000e+02 2.19518086e-02 2.00000000e+00]
 [5.72000000e+02 6.23000000e+02 2.25129661e-02 2.000000

In order to execute an automatic parenthood assertion, rules must be defined to organaize the process of extraction of the relation "is_a":
 1. All terms are concepts
 2. Agglomerative clustering of concepts
 3. A cluster is a concept
 4. A cluster concept of concepts $x$ and $y$ is going to have as term:
    1. Either find common synonim
    2. Either find if they are hyponyms or hypernyms of each other
    3. Either check for similar head
    4. Either don't give a name and group hopping that in a higher cluster a new concept will be found
 5. As for stopping condition, either stop when the distance between clusters is past threshold or when it gets to the top

### Automatic Parenthood Assertion

#### Rules

#### Implementation

In [26]:
from lib.ontologyextraction.schema import Term, Concept, Concept_Taxonomy

In [27]:
for term in filtered_list_of_terms:
    term = termEnrichment(term)

/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('communication.n.02') at depth 6
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('entity.n.01') at depth 10
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at depth 7
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('physical_entity.n.01') at dep

In [28]:
# Concept Creation
list_of_concepts = []
for term in filtered_list_of_terms:
    concept = Concept(term.term_name,[term],descriptive_term=term)
    list_of_concepts.append(concept)

In [29]:
conceptTaxonomy = Concept_Taxonomy(list_of_concepts)
conceptTaxonomy.createTaxonomyFromDistanceMatrix(Z)

/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('agent.n.03') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('hebrews.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('procession.n.02') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:604: UserWarning: Discarded redundant search for Synset('amerindian.n.01') at depth 5
  for synset 

In [30]:
final_concepts = []
for k in set(conceptTaxonomy.concept_dict.values()):
    if(len(k.concept_name) == 0):
        continue
    if k not in final_concepts:
        for x in final_concepts:
            if x.concept_name == k.concept_name and x in k.children_concept:
                final_concepts.remove(x)
        final_concepts.append(k)
print('The number of relevant deducted concepts are : ', len(final_concepts))

The number of relevant deducted concepts are :  308


In [31]:
highest_level = 0
for k in final_concepts:
    if k.level > highest_level:
        highest_level = k.level
print(highest_level)

18


In [32]:
for lv in range(highest_level):
    print('===========================================')
    print('===========================================')
    print('Level:' + str(lv))
    print('-------------------------------------------')
    for k in final_concepts:
        if k.level == lv:
            print(k.concept_name)

Level:0
-------------------------------------------
Level:1
-------------------------------------------
Level:2
-------------------------------------------
virus
antiviral agent
triphosphate
carbon
species
http
reaction
control
acyclovir
entity
agreement
surface
link
dss
division
framework
attachment
reaction
project
public
entity
real-time
new
infect
background
dengue
psychological feature
procedure
patent
countermeasures
biological
former
myocarditis
shock
civilization
nasopharyngeal
record
situation
psychological feature
operator
psychological feature
outbreak
wildlife
attribute
act
sanction
diseases
inactivation
correspondent
tissue
chemical
authority
entity
emerging
syndrome
epidemic
harvest
coronavirus
quality
sars
reporter
tropism
prevention
step
ibv-qx
virus
email
editor
disclosure
hex
night
risk
size
tract
asia
medicine
online
psychological feature
cqds
bat
pathotype
world
publication
study
physical entity
psychological feature
psychological feature
enzyme
rsv
abstraction
url


In [78]:
for concept in final_concepts:
    print('===========================================')
    print('===========================================')
    print('concept name :' + concept.concept_name)
    print('concept lv :' + str(concept.level))
    print('concept nb children :' + str(len(concept.children_concept)))
    print('-------------------------------------------')
    list_of_concept_names = list(set([i.concept_name for i in concept.children_concept]))
    for t in list_of_concept_names:
        if(len(t) > 0):
            print(t)

concept name :virus
concept lv :3
concept nb children :3
-------------------------------------------
parainfluenza
human parainfluenza virus
parainfluenza virus
concept name :nucleotide
concept lv :3
concept nb children :2
-------------------------------------------
oligonucleotide
nucleotide derivative
concept name :virus
concept lv :2
concept nb children :2
-------------------------------------------
encephalitis virus
japanese encephalitis virus
concept name :antiviral agent
concept lv :2
concept nb children :0
-------------------------------------------
concept name :triphosphate
concept lv :2
concept nb children :2
-------------------------------------------
labeled diheoxyribonucleotide triphosphate
diheoxyribonucleotide triphosphate
concept name :aminophenylboronic acid without any further modification
concept lv :4
concept nb children :1
-------------------------------------------
concept name :entity
concept lv :9
concept nb children :28
---------------------------------------

In [34]:
all_lv3_concepts = []
for k in final_concepts:
    if k.level <= 3:
        all_lv3_concepts.append(k)

In [35]:
for concept in all_lv3_concepts:
    print('===========================================')
    print('===========================================')
    print('concept name :' + concept.concept_name)
    print('concept lv :' + str(concept.level))
    print('-------------------------------------------')
    list_of_concept_names = list(set([i.concept_name for i in concept.children_concept]))
    print(len(concept.children_concept))
    for t in list_of_concept_names:
        if(len(t) > 0):
            print(t)

concept name :virus
concept lv :3
-------------------------------------------
3
parainfluenza
human parainfluenza virus
parainfluenza virus
concept name :nucleotide
concept lv :3
-------------------------------------------
2
oligonucleotide
nucleotide derivative
concept name :virus
concept lv :2
-------------------------------------------
2
encephalitis virus
japanese encephalitis virus
concept name :antiviral agent
concept lv :2
-------------------------------------------
0
concept name :triphosphate
concept lv :2
-------------------------------------------
2
labeled diheoxyribonucleotide triphosphate
diheoxyribonucleotide triphosphate
concept name :instagram
concept lv :3
-------------------------------------------
1
concept name :telephone
concept lv :3
-------------------------------------------
2
internet
address
concept name :physical entity
concept lv :3
-------------------------------------------
2
main
concept name :reagent
concept lv :3
---------------------------------------

### Final Hierarchy

In [149]:
def checkkid(concept1, concept2):
    if concept2.children_concept is None or len(concept2.children_concept) == 0:
        return False
    elif concept1 in concept2.children_concept:
        return True
    else:
        for concept in concept2.children_concept:
            return checkkid(concept1, concept)

def append_children(concept):
    if concept.children_concept is None or len(concept.children_concept) == 0:
        return [concept]
    else:
        list_of_concepts = []
        for concept1 in concept.children_concept:
            list_of_concepts += append_children(concept1)
        list_of_concepts.append(concept)
        return list_of_concepts

In [150]:
concept_hierachy = []
list_remove_later = []
for k in set(conceptTaxonomy.deducted_concepts):
    if(len(k.concept_name) == 0):
        continue
    already_in = False
    for x in concept_hierachy:
        if checkkid(x,k):
            list_remove_later.append(x)
        elif checkkid(k,x):
            already_in=True
    if not already_in and k not in concept_hierachy:
        concept_hierachy.append(k)

for k in list_remove_later:
    if k in concept_hierachy:
        concept_hierachy.remove(k)
print('The number high level concepts are : ', len(concept_hierachy))

The number high level concepts are :  134


In [165]:
def print_hierarchy(concept):
    def print_sub_hierarchy(concept1):
        print('-lv: '+ str(concept1.level) + ', name : ' + concept1.concept_name)
        for c in concept1.children_concept:
            print(print_sub_hierarchy(c))
    print('===========================================')
    print('===========================================')
    print('concept name :' + concept.concept_name)
    print('concept lv :' + str(concept.level))
    print('-------------------------------------------')
    for conceptc in concept.children_concept:
        print_sub_hierarchy(conceptc)
        print('-------------------------------------------')
    

In [172]:
count = 0
for concept in concept_hierachy:
    count += len(concept.children_concept)
    if(len(concept.concept_name) > 0):
        print_hierarchy(concept)

concept name :virus
concept lv :3
-------------------------------------------
-lv: 1, name : parainfluenza virus
-------------------------------------------
-lv: 1, name : parainfluenza
-------------------------------------------
-lv: 1, name : human parainfluenza virus
-------------------------------------------
concept name :entity
concept lv :9
-------------------------------------------
-lv: 1, name : lack
-------------------------------------------
-lv: 1, name : kidney
-------------------------------------------
-lv: 1, name : animal
-------------------------------------------
-lv: 2, name : risk
-lv: 1, name : increased risk
None
-lv: 1, name : risk
None
-------------------------------------------
-lv: 1, name : landscape
-------------------------------------------
-lv: 2, name : size
-lv: 1, name : bat sample size
None
-lv: 1, name : size
None
-------------------------------------------
-lv: 2, name : world
-------------------------------------------
-lv: 1, name : opportunity


## Non-taxonomic Relation Extraction

In [36]:
corpus = list(set(df['documentURL'].dropna().tolist()))
X_freq = np.zeros((len(corpus), len(labellist)))
word_list_dict = dict(zip(labellist,range(len(labellist))))
doc_dict = dict(zip(corpus,range(len(corpus))))

In [37]:
for label in labellist:
    list_of_docs = df.loc[df['Term']==label]['documentURL'].dropna().tolist()
    for doc in list_of_docs:
        X_freq[doc_dict[doc], word_list_dict[label]] = 1

In [38]:
df_corpus = pd.DataFrame(np.clip(X_freq,0,1), columns=labellist)

In [75]:
from mlxtend.frequent_patterns import apriori
frequent_itemsets = apriori(df_corpus, min_support = 0.5, max_len=2, use_colnames = True)

/home/alencga1/Documents/PreDiViD/git/predivid/code/IE/ievenv/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [76]:
from mlxtend.frequent_patterns import association_rules

relations = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
relations = relations[['antecedents','consequents']]
display(relations)

,antecedents,consequents
0,(http),(information)
1,(editor),(information)
2,(newsrx),(information)
3,(information),(virus)
4,(report),(information)
5,(university),(information)
6,(more information),(information)
7,(fact-based news),(information)
8,(article),(information)
9,(quote),(information)
